## HW 6
#### In this homework you will become the worlds most basic quantitative analysts. We will be looking at historical S&P data to gain intution on how long it takes for the stock market to hit a low after a high during a crash.
#### The idea is to buy during the low to get the best value for investing.
#### Note: please do not mistake this for actual investing advice, it is purely for intiution
#### Note2: This assignment was motivated by me trying to figure out what month of 2019 I should put money into my IRA.

## Part 1: Get Data in the Proper Format

In [ ]:
#a Import numpy, pandas, matplot as how it was done in class
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#b Read in the data; it is the file stocks.txt. Be careful about the seperator, it isn't a comma, it is a tab
#Call your data stocks
stocks = pd.read_csv('stocks.txt',sep='\t')

In [ ]:
#c There is an issue with one of the comma labels having an extra space, fix it
stocks.columns = ['Date','Value']

In [ ]:
#d We want to get rid of the day in the 'Date' column as we are most interested in months and years
#NO CODE SHOULD BE WRITTEN IN THIS CELL

In [ ]:
#di Create a series called months that only includes months and a series called years that only includes years
#Hint: access the string methods on stocks.dates, remember to expand
dates = stocks.Date.str.split(' ',expand=True)
months = dates[0]
years = dates[2]

In [ ]:
#dii Create a series called new_dates where each entry is `month` , `year`
#ex: Jan, 2019 (this will be the first entry)
#Hint: years may need to be converted to a str series before this is possible
new_dates = months + ', ' + years.astype(str)

In [ ]:
#diii Set the Date column in stocks as new_dates
stocks.Date = new_dates

In [ ]:
#div We need python to recognizes that our datatime truly are dates
#We can convert this series to a DateTime series
#Note, this will append back the 'day' that we just got rid of, but it will set each date as the 1st which is what we wanted
#Hint: look into the to_datetime function in pandas
stocks.Date = pd.to_datetime(stocks.Date)

In [ ]:
#e set the index of stocks to be the Date column
stocks.set_index('Date',inplace=True)

In [ ]:
#f get rid of the comma in stocks.Value and convert stocks.Value to a float series
#This will make sure the python recognizes we are working with numbers
stocks.Value = stocks.Value.str.replace(',','').astype(float)

## Part 2: Plot the dataset

In [ ]:
#We will check out the history of the S&P and improve our plot at each iteration
#NO CODE SHOULD BE WRITTEN IN THIS CELL

In [ ]:
#a Access the numpy array under the Value column, call it prices
prices = stocks.Value.values

In [ ]:
#b, plot prices using matplot
plt.plot(prices)

In [ ]:
#c, Notice that our plot is going backwards. We do not want to look backwards, in time, we want to look forwards.
#Reverse the order of prices, calling it prices_rev and plot that
prices_rev = prices[::-1]
plt.plot(prices_rev)

In [ ]:
#d, thats better but the growth of the market makes it hard to see trends before time point '1250'
#Lets try a plot using a log scale on the y axis
plt.semilogy(prices_rev)

In [ ]:
#e We worked so hard to get the dates column in the proper format, now lets use it in our plot
#NO CODE SHOULD BE WRITTEN IN THIS CELL

In [ ]:
#ei reverse the order of the underling dates numpy array and call it dates_rev
#Hint, you can access the index of stocks by using stocks.index
dates_rev = stocks.index.values[::-1]

In [ ]:
#eii, plot dates_rev with prices_rev
#Remember to still use the log scale on the y axis
plt.semilogy(dates_rev,prices_rev)

In [ ]:
#f, Give the plot and x label, a y label (make sure to include the fact that the y label is logged) and a title
#save it as plot.png
#show it
plt.semilogy(dates_rev,prices_rev)
plt.title('S&P Over the Years')
plt.ylabel('Logged Price')
plt.xlabel('Years')
plt.savefig('plot.png')
plt.show()

## Part 2b: Plotting the differenced series

In [ ]:
#a first, lets just reverse our entire dataframe so that we will get it in chronological order instead of 
#reverse chronologic order, call it stocks_rev
#Note: I could have had you do this earlier but thought it was more instructive to have it done here
#Hint, use .iloc to do array operations on our pandas dataframe
stocks_rev = stocks.iloc[::-1]

In [ ]:
#b Lets take a look at the differenced time series
#Use a certain pandas function to look at the single differenced time series of stocks_rev and called it stocks_diff
stocks_diff = stocks_rev.diff()

#c Which value becomes a NaN? Why?
Explain why here: 1871-01-01 does as we cannot take the difference of the first value in the series

In [ ]:
#d get rid of of the NaN value in stocks.diff
stocks_diff = stocks_diff.iloc[1:]
#or
stocks_diff = stocks_diff.dropna()

In [ ]:
#e, plot stocks_diff, we cannot use the log y as we have negative values,
#instead use symlog, this allows to do postive and negative y
#Note: matplot lib understands that the index is the x label and value is the y label

#FREE POINTS
plt.plot(stocks_diff)
plt.yscale('symlog')

In [ ]:
#f, this looks terrible, lets not try to connect our points and just plot as a scatter plot.
#Have each point labeled with an 'o'
plt.plot(stocks_diff,'o')
plt.yscale('symlog')

In [ ]:
#f, Give the plot and x label, a y label (make sure to include the fact that the y label is symlogged) and a title
#save it as diff_plot.png
#show it
plt.plot(stocks_diff,'o')
plt.yscale('symlog')
plt.title('S&P Changes Over the Years')
plt.ylabel('SymLogged Price')
plt.xlabel('Years')
plt.savefig('diff_plot.png')
plt.show()

# Part 3: Data Analysis
Part 2b was not that helpful. That is part of the data analysis process. Sometimes you do things that you think may help and they really don't. We could continue down the path of 2b, perhaps looking at differences in year averages or the the second differenced time series (the differences of the differences). Feel free to at your leisure, this is not the path we'll go down. 

In [ ]:
#For simplicity, lets say that the modern stock market begins after WWII in 1950.
#a Take the dates from 1950 to now in stocks_rev and call it stocks_mod
#Hint: use stocks_rev.loc['start':] were start will be the first date in 1950
stocks_mod = stocks_rev.loc['1950-01-01':]

In [ ]:
#b Lets smooth out some of the noise in our series by taking time windows 5 months at a time
#Then taking the mean of this window and labeling by the center
#Call it stocks_roll
#For example, what we call May 2018 will be the the mean of March, April, May, June, July 2018
#Hint: look here https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html
stocks_roll = stocks_mod.rolling(5,center=True).mean()
#Note: If you cannot get this part, you can download this dataset as stocks_rolling.csv from blackboard

In [ ]:
#c drop nans from stocks_roll 
#(we cannot have windows in the beggining and end as there is not enough data to take the mean of 5 points)
stocks_roll = stocks_roll.dropna()

In [ ]:
#d Do a log_y plot of stocksroll
plt.semilogy(stocks_roll)

In [ ]:
#Much smoother. 
#e Now we want to look for dips (local mins) in the market. A dip would be a month that is lower than both the month before
#And much after
#Get a DatetimeIndex of dips from stocks_roll and call its dips_list
#Hint: there are a few ways to do this, you can look at the differenced series in both directions, use .diff(-1) to get forward differences
#You can also do a rolling min in a length 3 window and see if your month is the rolling min

inds = (stocks_roll == stocks_roll.rolling(5,center=True).min()).values.squeeze()
dips_list = stocks_roll.index[inds]

#Or

inds = ((stocks_roll.diff(1)<0) & (stocks_roll.diff(-1) < 0)).values.squeeze()
dips_list = stocks_roll.index[inds]

In [ ]:
#f Do the semilog plot of stocks_roll again and look at dips list
plt.semilogy(stocks_roll)
print(dips_list)
plt.show()

* Look at 1950-05-01, clearly this is not a big enough dip to call a recession.
* We need some criterion on how much lower in price this month is compared to its neighbors

* Lets look at how much lower in percentge each "dip" is compared to the month before and after and then we can decide on a criterion
* Note: I say we but its really just me chooing this, however you can feel free to experiment with your own thresholds.

In [ ]:
#gi Create a dataframe that is the month to month percentage change in stocks_roll
#Note: there is a pandas methods for this, google it
#call it stocks_pct
stocks_pct = stocks_roll.pct_change()

In [ ]:
#gii multiply everything by 100 to get actual percentages
stocks_pct*=100

In [ ]:
#giii drop nans
stocks_pct = stocks_pct.dropna()

In [ ]:
#h Use dips_list to get pct_change only for dips
#Call this new dataframe, stocks_pct_need
stocks_pct_need =stocks_pct.loc[dips_list]

In [ ]:
#i Look at stocks_pct_needed in numerical order
#Note, you will need for first access the 'Value' series to use the sort_values method
stocks_pct_need.Value.sort_values()

In [ ]:
#j Look at the logged y plot of stocks_roll
plt.semilogy(stocks_roll)

In [ ]:
#k A reasonable threshold may be to only take dips that have a perctange decreae below -.7%
#Access only these dips from #dip_list and stock_pct_need, call it dips_need
dips_need = dips_list[(stocks_pct_need<-.7).values.squeeze()]


In [ ]:
#l We could do further analysis on the peaks but lets wrap up soon.
#Lets say assume that peaks happen (on average) exactly in the middle of dips
#Ex: if there was a dip in Jan 2020 and Jan 2018 then there would be a peak in Jan 2019
#Make a data_frame of dips_need and then take the differenced series and find the average time in between dips
#Note, we can do math operations on datatime objects
#Call this average as mean_time
mean_time = pd.DataFrame(dips_need).diff().mean()

In [ ]:
#m Take 1/2 of the mean_time to represent the time until next dip from each peak
#Call, this time_to_dip
time_to_dip = mean_time/2

In [ ]:
#n If there was a peak in October 2018, when should I invest in the dip?
#Hint: create an a datetime object using pd.datetime('08-13-2018') and then add time_to_dip
pd.to_datetime('08-13-2018') + time_to_dip